# Main Document Code Pipeline

## Importing relevant packages

In [108]:
import os
import gc
import math
import random
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-v0_8")
sns.set_context("notebook")
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    mean_squared_error,
    mean_absolute_error
)
from tqdm.auto import tqdm
import warnings


In [109]:
df = pd.read_csv("clean_NN_data.csv")

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212770 entries, 0 to 212769
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      212770 non-null  int64  
 1   station_id      212770 non-null  int64  
 2   hour            212749 non-null  object 
 3   departures      212770 non-null  float64
 4   arrivals        212770 non-null  float64
 5   hour_of_day     212770 non-null  int64  
 6   day_of_week     212770 non-null  int64  
 7   is_weekend      212770 non-null  int64  
 8   hour_sin        212770 non-null  float64
 9   hour_cos        212770 non-null  float64
 10  lat             212770 non-null  float64
 11  lon             212770 non-null  float64
 12  date            212749 non-null  object 
 13  temperature_2m  212749 non-null  float64
 14  precipitation   212749 non-null  float64
 15  rain            212749 non-null  float64
 16  snowfall        212749 non-null  float64
 17  snow_depth

In [111]:
df

,Unnamed: 0,station_id,hour,departures,arrivals,hour_of_day,day_of_week,is_weekend,hour_sin,hour_cos,lat,lon,date,temperature_2m,precipitation,rain,snowfall,snow_depth,weather_code,wind_speed_10m
0,0,147,2016-03-31 03:00:00+00:00,0.0,2.0,23,2,0,-2.588190e-01,9.659258e-01,40.715422,-74.011220,2016-03-31 03:00:00+00:00,5.9335,0.0,0.0,0.00,0.00,1.0,19.592731
1,1,152,2017-02-23 11:00:00+00:00,0.0,2.0,6,3,0,1.000000e+00,6.123234e-17,40.714740,-74.009106,2017-02-23 11:00:00+00:00,3.7225,0.0,0.0,0.00,0.00,0.0,6.489992
2,2,152,2017-02-24 11:00:00+00:00,0.0,2.0,6,4,0,1.000000e+00,6.123234e-17,40.714740,-74.009106,2017-02-24 11:00:00+00:00,9.8225,0.0,0.0,0.00,0.00,3.0,9.511088
3,3,152,2017-02-27 11:00:00+00:00,0.0,2.0,6,0,0,1.000000e+00,6.123234e-17,40.714740,-74.009106,2017-02-27 11:00:00+00:00,-0.7775,0.0,0.0,0.00,0.00,0.0,11.792404
4,4,152,2017-02-28 11:00:00+00:00,0.0,2.0,6,1,0,1.000000e+00,6.123234e-17,40.714740,-74.009106,2017-02-28 11:00:00+00:00,4.0725,0.0,0.0,0.00,0.00,3.0,2.620839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212765,212765,3426,2017-02-24 15:00:00+00:00,0.0,2.0,10,4,0,5.000000e-01,-8.660254e-01,40.709651,-74.068601,2017-02-24 15:00:00+00:00,16.3725,0.0,0.0,0.00,0.00,2.0,13.378250
212766,212766,3426,2017-03-05 17:00:00+00:00,0.0,4.0,12,6,1,1.224647e-16,-1.000000e+00,40.709651,-74.068601,2017-03-05 17:00:00+00:00,-1.1275,0.0,0.0,0.00,0.00,0.0,19.513195
212767,212767,3426,2017-03-10 14:00:00+00:00,0.0,4.0,9,4,0,7.071068e-01,-7.071068e-01,40.709651,-74.068601,2017-03-10 14:00:00+00:00,0.7725,1.4,0.0,0.98,0.01,75.0,17.555307
212768,212768,3426,2017-03-31 13:00:00+00:00,0.0,4.0,9,4,0,7.071068e-01,-7.071068e-01,40.709651,-74.068601,2017-03-31 13:00:00+00:00,3.4725,0.8,0.8,0.00,0.00,53.0,11.367109


### Setting Initial features (before lag)

In [112]:
FEATURES = [
    "arrivals",
    "hour_of_day",
    "day_of_week",
    "is_weekend",
    "hour_sin",
    "hour_cos",
    "temperature_2m",
    "precipitation",
    "rain",
    "snowfall",
    "snow_depth",
    "weather_code",
    "wind_speed_10m",
]

TARGET = "departures"

df = df.copy()

df["hour"] = pd.to_datetime(df["hour"])
df = df.sort_values("hour")


### Creating grouped dataset with features

In [113]:
df_nn = df[
    ["station_id", "hour"] + FEATURES + [TARGET]
].dropna().reset_index(drop=True)


### Adding lag features

In [114]:
# Ensure non-negative departures (important for log1p)
df_nn["departures"] = df_nn["departures"].clip(lower=0)

# Sort for time-safe lagging
df_nn = df_nn.sort_values(["station_id", "hour"])

# Lagged features
df_nn["lag_1"]   = df_nn.groupby("station_id")["departures"].shift(1)
df_nn["lag_24"]  = df_nn.groupby("station_id")["departures"].shift(24)
df_nn["lag_168"] = df_nn.groupby("station_id")["departures"].shift(168)

df_nn["roll_24"] = (
    df_nn.groupby("station_id")["departures"]
    .shift(1)
    .rolling(24)
    .mean()
)

# 🚨 THIS LINE IS MANDATORY 🚨
df_nn = df_nn.dropna().reset_index(drop=True)

# Log-transform AFTER cleaning
df_nn["y"] = np.log1p(df_nn["departures"])


### Updating feature list

In [115]:
FEATURES = [
    "arrivals",
    "hour_of_day",
    "day_of_week",
    "is_weekend",
    "hour_sin",
    "hour_cos",
    "temperature_2m",
    "precipitation",
    "rain",
    "snowfall",
    "snow_depth",
    "weather_code",
    "wind_speed_10m",
    # lagged demand features
    "lag_1",
    "lag_24",
    "lag_168",
    "roll_24",
]


### Creating mapping bettwen stations

In [116]:
# Create mapping
station_ids = df_nn["station_id"].unique()
station_to_idx = {sid: i for i, sid in enumerate(station_ids)}

# Apply mapping
df_nn["station_idx"] = df_nn["station_id"].map(station_to_idx)


### Log Transfomring Predictor column to avoid heteroskedasticity

In [117]:
df_nn["y"] = np.log1p(df_nn[TARGET])


### Doing time based split since we are working with time series data

In [118]:
split_idx = int(len(df_nn) * 0.8)

train_df = df_nn.iloc[:split_idx]
val_df   = df_nn.iloc[split_idx:]


### Scaling datapoints

In [119]:
scaler = StandardScaler()

X_train = scaler.fit_transform(train_df[FEATURES].values)
X_val   = scaler.transform(val_df[FEATURES].values)

y_train = train_df["y"].values.reshape(-1, 1)
y_val   = val_df["y"].values.reshape(-1, 1)

station_train = train_df["station_idx"].values
station_val   = val_df["station_idx"].values



### Creating Neural Network Architecture

In [120]:
class DemandDataset(Dataset):
    def __init__(self, X, station_id, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.station_id = torch.tensor(station_id, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.station_id[idx], self.y[idx]


In [121]:
train_ds = DemandDataset(X_train, station_train, y_train)
val_ds   = DemandDataset(X_val, station_val, y_val)

train_loader = DataLoader(train_ds, batch_size=512, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=512, shuffle=False)


In [122]:
class DemandNN(nn.Module):
    def __init__(self, num_stations, num_features, emb_dim=8):
        super().__init__()

        self.station_emb = nn.Embedding(num_stations, emb_dim)

        self.net = nn.Sequential(
            nn.Linear(num_features + emb_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x, station_id):
        emb = self.station_emb(station_id)
        x = torch.cat([x, emb], dim=1)
        return self.net(x)


In [123]:
device = "cuda" if torch.cuda.is_available() else "cpu"

num_stations = df_nn["station_id"].nunique()
num_features = len(FEATURES)

model = DemandNN(num_stations, num_features).to(device)


In [124]:
criterion = nn.HuberLoss(delta=1.0)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-5
)


### Define training and evaluation loop

In [125]:

def train_epoch(model, loader):
    model.train()
    total_loss = 0.0

    for X, station_id, y in loader:
        X = X.to(device)
        station_id = station_id.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        preds = model(X, station_id)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * len(X)

    return total_loss / len(loader.dataset)

def eval_epoch(model, loader):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for X, station_id, y in loader:
            X = X.to(device)
            station_id = station_id.to(device)
            y = y.to(device)

            preds = model(X, station_id)
            loss = criterion(preds, y)
            total_loss += loss.item() * len(X)

    return total_loss / len(loader.dataset)



### Training the model

In [126]:
EPOCHS = 30

for epoch in range(1, EPOCHS + 1):
    train_loss = train_epoch(model, train_loader)
    val_loss = eval_epoch(model, val_loader)

    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f}"
    )


Epoch 01 | Train Loss: 0.2938 | Val Loss: 0.2440
Epoch 02 | Train Loss: 0.2058 | Val Loss: 0.1984
Epoch 03 | Train Loss: 0.1838 | Val Loss: 0.1895
Epoch 04 | Train Loss: 0.1785 | Val Loss: 0.1875
Epoch 05 | Train Loss: 0.1761 | Val Loss: 0.1870
Epoch 06 | Train Loss: 0.1742 | Val Loss: 0.1898
Epoch 07 | Train Loss: 0.1731 | Val Loss: 0.1888
Epoch 08 | Train Loss: 0.1720 | Val Loss: 0.1917
Epoch 09 | Train Loss: 0.1711 | Val Loss: 0.1905
Epoch 10 | Train Loss: 0.1708 | Val Loss: 0.1992
Epoch 11 | Train Loss: 0.1702 | Val Loss: 0.1967
Epoch 12 | Train Loss: 0.1697 | Val Loss: 0.1935
Epoch 13 | Train Loss: 0.1693 | Val Loss: 0.1928
Epoch 14 | Train Loss: 0.1688 | Val Loss: 0.1948
Epoch 15 | Train Loss: 0.1685 | Val Loss: 0.1936
Epoch 16 | Train Loss: 0.1680 | Val Loss: 0.1946
Epoch 17 | Train Loss: 0.1677 | Val Loss: 0.1971
Epoch 18 | Train Loss: 0.1677 | Val Loss: 0.1946
Epoch 19 | Train Loss: 0.1675 | Val Loss: 0.1986
Epoch 20 | Train Loss: 0.1670 | Val Loss: 0.1948
Epoch 21 | Train Los

### Getting a set of predictions to ensure it can be swtich back in interms of number of bikes

In [128]:
model.eval()

with torch.no_grad():
    X_sample = torch.tensor(X_val[:10], dtype=torch.float32).to(device)
    s_sample = torch.tensor(station_val[:10], dtype=torch.long).to(device)

    log_preds = model(X_sample, s_sample).cpu().numpy().flatten()
    preds = np.expm1(log_preds)

preds


array([0.21395354, 0.08630245, 2.6327817 , 2.978732  , 3.0221467 ,
       3.130514  , 0.9277921 , 1.1133342 , 1.2486101 , 2.6773984 ],
      dtype=float32)

In [129]:
y_val_pred_log = []
y_val_true_log = []

model.eval()
with torch.no_grad():
    for X, station_id, y in val_loader:
        X = X.to(device)
        station_id = station_id.to(device)

        preds = model(X, station_id)
        y_val_pred_log.append(preds.cpu().numpy())
        y_val_true_log.append(y.numpy())

y_val_pred_log = np.vstack(y_val_pred_log).flatten()
y_val_true_log = np.vstack(y_val_true_log).flatten()

# Back to real units
y_val_pred = np.expm1(y_val_pred_log)
y_val_true = np.expm1(y_val_true_log)


### Defining evaluation metrics for CV

In [133]:
def time_series_cv_splits(df, n_splits=5, min_train_frac=0.5):
    """
    Generator yielding (train_idx, val_idx) for rolling-origin CV
    """
    n = len(df)
    train_end = int(n * min_train_frac)
    step = (n - train_end) // n_splits

    for i in range(n_splits):
        train_idx = np.arange(0, train_end + i * step)
        val_idx = np.arange(train_end + i * step,
                             min(train_end + (i + 1) * step, n))
        yield train_idx, val_idx


In [134]:
def compute_metrics(y_true, y_pred):
    mae = np.mean(np.abs(y_pred - y_true))
    rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))

    mask = y_true > 1
    mape = np.mean(
        np.abs(y_pred[mask] - y_true[mask]) / y_true[mask]
    )

    return {
        "MAE": mae,
        "RMSE": rmse,
        "MAPE": mape
    }


### Running cross validation on the dataset

In [135]:
cv_results = []

for fold, (train_idx, val_idx) in enumerate(time_series_cv_splits(df_nn)):

    train_df = df_nn.iloc[train_idx]
    val_df   = df_nn.iloc[val_idx]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(train_df[FEATURES].values)
    X_val   = scaler.transform(val_df[FEATURES].values)

    y_train = train_df["y"].values.reshape(-1, 1)
    y_val   = val_df["y"].values.reshape(-1, 1)

    s_train = train_df["station_idx"].values
    s_val   = val_df["station_idx"].values

    train_ds = DemandDataset(X_train, s_train, y_train)
    val_ds   = DemandDataset(X_val, s_val, y_val)

    train_loader = DataLoader(train_ds, batch_size=512, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=512, shuffle=False)

    model = DemandNN(
        num_stations=len(station_to_idx),
        num_features=len(FEATURES),
        emb_dim=8
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    criterion = nn.HuberLoss(delta=1.0)

    # Train briefly (early stopping in practice)
    for _ in range(10):
        train_epoch(model, train_loader)

    # Predict on validation
    model.eval()
    preds_log = []
    true_log = []

    with torch.no_grad():
        for X, s, y in val_loader:
            X, s = X.to(device), s.to(device)
            preds_log.append(model(X, s).cpu().numpy())
            true_log.append(y.numpy())

    preds = np.expm1(np.vstack(preds_log).flatten())
    true  = np.expm1(np.vstack(true_log).flatten())

    metrics = compute_metrics(true, preds)
    metrics["fold"] = fold
    cv_results.append(metrics)


### Getting evvaluation metrics

In [136]:
cv_df = pd.DataFrame(cv_results)

cv_summary = cv_df.mean().to_dict()
cv_summary


{'MAE': 1.9143524169921875,
 'RMSE': 3.246798038482666,
 'MAPE': 0.5097813606262207,
 'fold': 2.0}